# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 9 2023 8:30AM,258964,10,MSP219298,"Methapharm, Inc.",Released
1,Mar 9 2023 8:30AM,258964,10,MSP219300,"Methapharm, Inc.",Released
2,Mar 9 2023 8:30AM,258964,10,MSP219305,"Methapharm, Inc.",Released
3,Mar 9 2023 8:30AM,258964,10,MSP219312,"Methapharm, Inc.",Released
4,Mar 9 2023 8:30AM,258964,10,MSP219313,"Methapharm, Inc.",Released
5,Mar 9 2023 8:30AM,258964,10,MSP219324,"Methapharm, Inc.",Released
6,Mar 9 2023 8:30AM,258977,10,9244916,"Methapharm, Inc.",Released
7,Mar 9 2023 8:30AM,258976,10,SNAP35340612,"Snap Medical Industries, LLC",Released
8,Mar 9 2023 8:30AM,258964,10,MSP219293,Methapharm-G,Released
9,Mar 8 2023 4:42PM,258978,22,643254,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
9,258964,Released,7
10,258975,Released,1
11,258976,Released,1
12,258977,Released,1
13,258978,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258964,NaN,7.0
258975,NaN,1.0
258976,NaN,1.0
258977,NaN,1.0
258978,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258896,3.0,2.0
258897,3.0,1.0
258904,1.0,0.0
258941,13.0,1.0
258953,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258896,3,2
258897,3,1
258904,1,0
258941,13,1
258953,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258896,3,2
1,258897,3,1
2,258904,1,0
3,258941,13,1
4,258953,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258896,3,2
1,258897,3,1
2,258904,1,
3,258941,13,1
4,258953,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 9 2023 8:30AM,258964,10,"Methapharm, Inc."
6,Mar 9 2023 8:30AM,258977,10,"Methapharm, Inc."
7,Mar 9 2023 8:30AM,258976,10,"Snap Medical Industries, LLC"
8,Mar 9 2023 8:30AM,258964,10,Methapharm-G
9,Mar 8 2023 4:42PM,258978,22,"NBTY Global, Inc."
10,Mar 8 2023 4:10PM,258975,19,"GUSA Granules USA, Inc."
11,Mar 8 2023 2:47PM,258960,19,"GUSA Granules USA, Inc."
12,Mar 8 2023 1:27PM,258953,22,"NBTY Global, Inc."
13,Mar 8 2023 11:32AM,258941,15,"Brookfield Pharmaceuticals, LLC"
27,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 9 2023 8:30AM,258964,10,"Methapharm, Inc.",,7
1,Mar 9 2023 8:30AM,258964,10,Methapharm-G,,7
2,Mar 9 2023 8:30AM,258977,10,"Methapharm, Inc.",,1
3,Mar 9 2023 8:30AM,258976,10,"Snap Medical Industries, LLC",,1
4,Mar 8 2023 4:42PM,258978,22,"NBTY Global, Inc.",,1
5,Mar 8 2023 4:10PM,258975,19,"GUSA Granules USA, Inc.",,1
6,Mar 8 2023 2:47PM,258960,19,"GUSA Granules USA, Inc.",,1
7,Mar 8 2023 1:27PM,258953,22,"NBTY Global, Inc.",,1
8,Mar 8 2023 11:32AM,258941,15,"Brookfield Pharmaceuticals, LLC",13,1
9,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",3,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 8:30AM,258964,10,"Methapharm, Inc.",7,
1,Mar 9 2023 8:30AM,258964,10,Methapharm-G,7,
2,Mar 9 2023 8:30AM,258977,10,"Methapharm, Inc.",1,
3,Mar 9 2023 8:30AM,258976,10,"Snap Medical Industries, LLC",1,
4,Mar 8 2023 4:42PM,258978,22,"NBTY Global, Inc.",1,
5,Mar 8 2023 4:10PM,258975,19,"GUSA Granules USA, Inc.",1,
6,Mar 8 2023 2:47PM,258960,19,"GUSA Granules USA, Inc.",1,
7,Mar 8 2023 1:27PM,258953,22,"NBTY Global, Inc.",1,
8,Mar 8 2023 11:32AM,258941,15,"Brookfield Pharmaceuticals, LLC",1,13
9,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",1,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 8:30AM,258964,10,"Methapharm, Inc.",7,
1,Mar 9 2023 8:30AM,258964,10,Methapharm-G,7,
2,Mar 9 2023 8:30AM,258977,10,"Methapharm, Inc.",1,
3,Mar 9 2023 8:30AM,258976,10,"Snap Medical Industries, LLC",1,
4,Mar 8 2023 4:42PM,258978,22,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 8:30AM,258964,10,"Methapharm, Inc.",7.0,NaN
1,Mar 9 2023 8:30AM,258964,10,Methapharm-G,7.0,NaN
2,Mar 9 2023 8:30AM,258977,10,"Methapharm, Inc.",1.0,NaN
3,Mar 9 2023 8:30AM,258976,10,"Snap Medical Industries, LLC",1.0,NaN
4,Mar 8 2023 4:42PM,258978,22,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 8:30AM,258964,10,"Methapharm, Inc.",7.0,0.0
1,Mar 9 2023 8:30AM,258964,10,Methapharm-G,7.0,0.0
2,Mar 9 2023 8:30AM,258977,10,"Methapharm, Inc.",1.0,0.0
3,Mar 9 2023 8:30AM,258976,10,"Snap Medical Industries, LLC",1.0,0.0
4,Mar 8 2023 4:42PM,258978,22,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1035881,16.0,0.0
15,776734,4.0,19.0
19,517935,2.0,0.0
21,258904,0.0,1.0
22,517931,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1035881,16.0,0.0
1,15,776734,4.0,19.0
2,19,517935,2.0,0.0
3,21,258904,0.0,1.0
4,22,517931,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,16.0,0.0
1,15,4.0,19.0
2,19,2.0,0.0
3,21,0.0,1.0
4,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,16.0
1,15,Released,4.0
2,19,Released,2.0
3,21,Released,0.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21,22
Status,,,,,
Executing,0.0,19.0,0.0,1.0,0.0
Released,16.0,4.0,2.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21,22
0,Executing,0.0,19.0,0.0,1.0,0.0
1,Released,16.0,4.0,2.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21,22
0,Executing,0.0,19.0,0.0,1.0,0.0
1,Released,16.0,4.0,2.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()